In [1]:
#imports  

import pandas as pd
import os
import re
import numpy as np



In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_colwidth', 20)

In [3]:
# remove all letters and special characters from coordinates

def cleanCoordinates(coord):
  return re.sub(r'[A-Za-z()]+', '', coord).strip()


In [4]:
# convert the number to a string with the specified sig fits

def truncateString(number):
  if '.' in number:
    index = number.index('.')
    return number[:index + 6] 
  return number


In [5]:
# folderpath for the coordinates you are taking

folder_path = "G:/Shared drives/host_tree_cnn/merging_autoarborist_w_inventories/og_inventories_w_names_coords/"
if os.path.exists(folder_path) and os.path.isdir(folder_path):
  print("Folder exists.")
else:
  print("Folder does not exist or the path is incorrect.")


Folder exists.


In [6]:
#format the main dataframe
mainDF = pd.DataFrame(dtype=str)

# iterate through the coords document and append it to one dataframe
for i, filename in enumerate(os.listdir(folder_path)):
  file_path = os.path.join(folder_path, filename)
  print(file_path)
  cityDF = pd.read_csv(file_path, dtype=str)
    
  # do string operations for incorrect formatting
  if not {'rounded_lng', 'rounded_lat'}.issubset(cityDF.columns):
    cityDF['coordinates'] = cityDF['coordinate'].astype(str).apply(cleanCoordinates)
    cityDF[['rounded_lng', 'rounded_lat']] = cityDF['coordinates'].str.split(expand=True)
  
  # store the original lat/long for debugging purposes 
  cityDF['original_lng'] = cityDF['rounded_lng']
  cityDF['original_lat'] = cityDF['rounded_lat']

  #  take only the columns you need
  cityDF = cityDF[['original_lng', 'rounded_lng', 'original_lat', 'rounded_lat', 'genus_name', 'species_name']]
  

  mainDF = pd.concat([mainDF, cityDF], ignore_index=True)
  
mainDF['rounded_lng'] = mainDF['rounded_lng'].astype(str).apply(truncateString)
mainDF['rounded_lat'] = mainDF['rounded_lat'].astype(str).apply(truncateString)
mainDF.fillna('Missing', inplace=True)

G:/Shared drives/host_tree_cnn/merging_autoarborist_w_inventories/og_inventories_w_names_coords/bloomington_inventory.csv
G:/Shared drives/host_tree_cnn/merging_autoarborist_w_inventories/og_inventories_w_names_coords/boulder_inventory.csv
G:/Shared drives/host_tree_cnn/merging_autoarborist_w_inventories/og_inventories_w_names_coords/buffalo_inventory.csv
G:/Shared drives/host_tree_cnn/merging_autoarborist_w_inventories/og_inventories_w_names_coords/calgary_inventory.csv
G:/Shared drives/host_tree_cnn/merging_autoarborist_w_inventories/og_inventories_w_names_coords/cambridge_inventory.csv
G:/Shared drives/host_tree_cnn/merging_autoarborist_w_inventories/og_inventories_w_names_coords/charlottesville_inventory.csv
G:/Shared drives/host_tree_cnn/merging_autoarborist_w_inventories/og_inventories_w_names_coords/cupertino_inventory.csv
G:/Shared drives/host_tree_cnn/merging_autoarborist_w_inventories/og_inventories_w_names_coords/denver_inventory.csv
G:/Shared drives/host_tree_cnn/merging_au

In [7]:
# Generate the auto arborist data 
AAPath =  "G:/Shared drives/host_tree_cnn/merging_autoarborist_w_inventories/og_autoarborist/tree_locations_tfrecord_idx_merged.csv"
AADF  = pd.read_csv(AAPath, dtype=str)

# format the lat and long
AADF['rounded_lat'] = AADF['SHAPE_LAT']
AADF['rounded_lng']= AADF['SHAPE_LNG']
AADF['rounded_lng'] = AADF['rounded_lng'].apply(truncateString)
AADF['rounded_lat'] = AADF['rounded_lat'].apply(truncateString)

# complete a left merge 
AADF = pd.merge(AADF, mainDF[['rounded_lng', 'rounded_lat', 'original_lat', 'original_lng', 'genus_name', 'species_name']], 
                    on=['rounded_lng', 'rounded_lat'], 
                    how='left')
filename = "AutoArboristData.csv"
AADF.to_csv('G:/Shared drives/host_tree_cnn/merging_autoarborist_w_inventories/autoarborist_names_appended/' + filename, index=False)


In [8]:

totalRows = AADF.shape[0]

# missing - there is a lat/long match, but there is not corresponding genus
missingPercent = (AADF["genus_name"] == "Missing").mean() * 100

print(f"Percentage of 'Missing' values in 'genus_name' column: {missingPercent}%")


# there is no lat/long match
NaValues = AADF['genus_name'].isnull().sum() + (AADF['genus_name'] == '').sum()


percentNA = (NaValues / totalRows) * 100 if totalRows > 0 else 0

print(f"Percentage of NaN/empty values in 'genus_name' column: {percentNA:}%")


Percentage of 'Missing' values in 'genus_name' column: 4.842329566115402%
Percentage of NaN/empty values in 'genus_name' column: 25.88435077307966%


In [9]:
# Check the status of a certain city
cityDF = AADF[AADF["City"] == "Columbus"]
totalRows = cityDF.shape[0]
# total rows for the city
print(totalRows)

# percent missing (genus was not available for the coords)
missingPercent = (cityDF["genus_name"] == "Missing").mean() * 100
print(f"Percentage of 'Missing' values in 'genus_name' column: {missingPercent}%")


# percent absent (no coords match
NaValues = cityDF['genus_name'].isnull().sum() + (cityDF['genus_name'] == '').sum()
percentNA = (NaValues / totalRows) * 100 if totalRows > 0 else 0
print(f"Percentage of NaN/empty values in 'genus_name' column: {percentNA:}%")



74438
Percentage of 'Missing' values in 'genus_name' column: 2.74590934737634%
Percentage of NaN/empty values in 'genus_name' column: 84.95795158386845%


# 